In [ ]:
import os
import hashlib
import configparser
import requests
import tkinter as tk
from tkinter import filedialog, messagebox, Menu

# Configurare inițială pentru fișierul settings.ini
CONFIG_FILE = "settings.ini"
config = configparser.ConfigParser()

if not os.path.exists(CONFIG_FILE):
    config["settings"] = {
        "style": "Dark",
        "virustotalscan": "False",
        "virustotalapikey": "",
        "metadefenderscan": "False",
        "metadefenderapikey": "",
        "usehashfiles": "False"
    }
    with open(CONFIG_FILE, "w") as configfile:
        config.write(configfile)

# Citire configurări din fișier
config.read(CONFIG_FILE)

class VirusScanApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Predicție online pe bază de fișier/hash")
        self.root.geometry("800x600")

        # Inițializare variabile pentru setări
        self.use_hash_files_var = tk.BooleanVar(value=config.getboolean("settings", "usehashfiles"))
        self.use_virustotal_var = tk.BooleanVar(value=config.getboolean("settings", "virustotalscan"))
        self.use_metadefender_var = tk.BooleanVar(value=config.getboolean("settings", "metadefenderscan"))

        # Creare meniu
        self.create_menu()

        # Layout principal
        self.create_widgets()

    def create_menu(self):
        menu_bar = Menu(self.root)

        # Meniu de setări
        settings_menu = Menu(menu_bar, tearoff=0)
        settings_menu.add_checkbutton(label="Folosește fișiere locale de hash",
                                      variable=self.use_hash_files_var,
                                      command=self.update_settings)
        settings_menu.add_checkbutton(label="Folosește VirusTotal",
                                      variable=self.use_virustotal_var,
                                      command=self.update_settings)
        settings_menu.add_checkbutton(label="Folosește MetaDefender",
                                      variable=self.use_metadefender_var,
                                      command=self.update_settings)
        settings_menu.add_separator()
        settings_menu.add_command(label="Setări API", command=self.open_api_settings)
        menu_bar.add_cascade(label="Setări", menu=settings_menu)

        # Atașare meniu principal
        self.root.config(menu=menu_bar)

    def create_widgets(self):
        # Secțiunea de fișier
        self.file_button = tk.Button(self.root, text="Selectează fișier", command=self.load_and_scan_file)
        self.file_button.pack(padx=10, pady=10)

        # Secțiunea de rezultate
        self.results_text = tk.Text(self.root, height=20, width=80)
        self.results_text.pack(padx=10, pady=10)
        self.results_text.config(state=tk.DISABLED)

    def open_api_settings(self):
        # Fereastră nouă pentru setări API
        api_window = tk.Toplevel(self.root)
        api_window.title("Setări API")
        api_window.geometry("500x200")

        # VirusTotal API Key
        virustotal_label = tk.Label(api_window, text="VirusTotal API Key:")
        virustotal_label.grid(row=0, column=0, sticky="w", padx=10, pady=5)
        self.virustotal_api_entry = tk.Entry(api_window, width=50)
        self.virustotal_api_entry.insert(0, config.get("settings", "virustotalapikey"))
        self.virustotal_api_entry.grid(row=0, column=1, padx=10, pady=5)

        # MetaDefender API Key
        metadefender_label = tk.Label(api_window, text="MetaDefender API Key:")
        metadefender_label.grid(row=1, column=0, sticky="w", padx=10, pady=5)
        self.metadefender_api_entry = tk.Entry(api_window, width=50)
        self.metadefender_api_entry.insert(0, config.get("settings", "metadefenderapikey"))
        self.metadefender_api_entry.grid(row=1, column=1, padx=10, pady=5)

        # Buton pentru salvare
        save_button = tk.Button(api_window, text="Salvează", command=self.save_api_keys)
        save_button.grid(row=2, column=0, columnspan=2, pady=10)

    def save_api_keys(self):
        # Salvează noile chei API în fișierul settings.ini
        config.set("settings", "virustotalapikey", self.virustotal_api_entry.get())
        config.set("settings", "metadefenderapikey", self.metadefender_api_entry.get())

        with open(CONFIG_FILE, "w") as configfile:
            config.write(configfile)

        messagebox.showinfo("Succes", "Setările API au fost salvate cu succes!")

    def update_settings(self):
        # Actualizează stările checkbox-urilor în fișierul settings.ini
        config.set("settings", "usehashfiles", str(self.use_hash_files_var.get()))
        config.set("settings", "virustotalscan", str(self.use_virustotal_var.get()))
        config.set("settings", "metadefenderscan", str(self.use_metadefender_var.get()))

        with open(CONFIG_FILE, "w") as configfile:
            config.write(configfile)


    def load_and_scan_file(self):
        """
        Selectează un fișier, calculează hash-ul și verifică în fișiere locale și API-uri.
        """
        file_path = filedialog.askopenfilename(title="Selectează un fișier")
        if not file_path:
            return

        file_name = os.path.basename(file_path)  # Obține numele fișierului
        self.display_result(f"Numele fișierului: {file_name}")

        # Calculăm SHA-256 pentru fișierul selectat
        file_hash = self.calculate_sha256(file_path)
        self.display_result(f"Hash calculat (SHA-256): {file_hash}")

        # Verificăm hash-ul în fișiere locale și API-uri
        self.verify_hash(file_hash, file_path)

    def verify_hash(self, file_hash, file_path=None):
        """
        Verifică hash-ul în fișierele locale și API-uri.
        """
        # Verificăm hash-ul în fișierele locale
        self.display_result("\nVerificare hash în fișierele hash:")
        hash_files = ["./hard_signatures/SHA256-Hashes_pack1.txt",
                    "./hard_signatures/SHA256-Hashes_pack2.txt",
                    "./hard_signatures/SHA256-Hashes_pack3.txt"]
        hash_found = False

        for hash_file in hash_files:
            self.display_result(f"Verificare în {os.path.basename(hash_file)}:")
            result = self.check_hash_in_file(file_hash, hash_file)
            self.display_result(result)
            if "a fost găsit" in result:
                hash_found = True

        if not hash_found:
            self.display_result("Hash-ul NU a fost găsit în niciun fișier de hash-uri.")

        # Verificare cu VirusTotal
        if self.use_virustotal_var.get():
            self.display_result("\nVerificare cu VirusTotal:")
            vt_result = self.check_virustotal(file_hash, file_path)
            self.display_result(vt_result)

            # Dacă există un ID de analiză, obținem detaliile
            if "Analysis ID:" in vt_result:
                analysis_id = vt_result.split("Analysis ID:")[1].strip()
                detailed_vt_result = self.get_virustotal_analysis(analysis_id)
                self.display_result(f"Detalii analiză VirusTotal:\n{detailed_vt_result}")

        # Verificare cu MetaDefender
        if self.use_metadefender_var.get():
            self.display_result("\nVerificare cu MetaDefender:")
            md_result = self.check_metadefender(file_hash, file_path)
            self.display_result(md_result)

            # Dacă există un Data ID, obținem detaliile
            if "Data ID:" in md_result:
                data_id = md_result.split("Data ID:")[1].strip()
                detailed_md_result = self.get_metadefender_analysis(data_id)
                self.display_result(f"Detalii analiză MetaDefender:\n{detailed_md_result}")



    def calculate_sha256(self, file_path):
        """
        Calculează hash-ul SHA-256 al unui fișier.
        """
        sha256_hash = hashlib.sha256()
        try:
            with open(file_path, "rb") as f:
                for byte_block in iter(lambda: f.read(4096), b""):
                    sha256_hash.update(byte_block)
            return sha256_hash.hexdigest()
        except Exception as e:
            self.display_result(f"Eroare la calcularea hash-ului: {e}")
            return ""

    def check_hash_in_file(self, file_hash, hash_file_path):
        """
        Verifică dacă hash-ul dat se află într-un fișier de hash-uri.
        """
        if not os.path.exists(hash_file_path):
            return f"Fișierul {hash_file_path} nu există."

        try:
            with open(hash_file_path, "r") as hash_file:
                for line in hash_file:
                    if file_hash in line.strip():
                        return "Hash-ul a fost găsit în lista de hash-uri!"
            return "Hash-ul NU a fost găsit în lista de hash-uri."
        except Exception as e:
            return f"Eroare la citirea fișierului: {e}"


    def display_result(self, text):
            self.results_text.config(state=tk.NORMAL)
            self.results_text.insert(tk.END, text + "\n")
            self.results_text.config(state=tk.DISABLED)

    def calculate_sha256(self, file_path):
            sha256_hash = hashlib.sha256()
            with open(file_path, "rb") as f:
                for byte_block in iter(lambda: f.read(4096), b""):
                    sha256_hash.update(byte_block)
            return sha256_hash.hexdigest()
    
    def check_virustotal(self, file_hash, file_path=None):
        """
        Verifică un hash sau încarcă un fișier în VirusTotal.
        """
        api_key = config.get("settings", "virustotalapikey")
        if not api_key:
            return "Cheia API VirusTotal nu este configurată."

        headers = {"x-apikey": api_key}
        url = f"https://www.virustotal.com/api/v3/files/{file_hash}"

        try:
            # Verificăm hash-ul în baza de date
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                # Parsăm rezultatele
                data = response.json().get("data", {}).get("attributes", {})
                stats = data.get("last_analysis_stats", {})
                malicious = stats.get("malicious", 0)
                harmless = stats.get("harmless", 0)
                suspicious = stats.get("suspicious", 0)

                # Detalii pentru fiecare motor antivirus
                engines = data.get("last_analysis_results", {})
                engine_details = ""
                for engine, result in engines.items():
                    status = result.get("category", "Unknown")
                    engine_details += f"{engine}: {status}\n"

                detail = f" - Malicious: {malicious}\n - Harmless: {harmless}\n - Suspicious: {suspicious}\n"
                detail += f" - Scan date: {data.get('last_analysis_date', 'N/A')}\n"
                detail += f" - Detalii analize motor antivirus:\n{engine_details}"

                return detail
            elif response.status_code == 404 and file_path:
                # Dacă hash-ul nu este găsit, încărcăm fișierul
                self.display_result("Hash-ul nu există. Încercare de încărcare a fișierului...")
                return self.upload_to_virustotal(file_path)
            else:
                # Alte erori
                return f"Eroare la interogarea VirusTotal. Status code: {response.status_code}\nRăspuns: {response.text}"
        except Exception as e:
            return f"Eroare la interogarea VirusTotal: {e}"



    def check_metadefender(self, file_hash, file_path=None):
        """
        Verifică un hash sau încarcă un fișier în MetaDefender, returnând linkul și data_id.
        """
        api_key = config.get("settings", "metadefenderapikey")
        if not api_key:
            return "Cheia API MetaDefender nu este configurată."

        headers = {"apikey": api_key}
        url = f"https://api.metadefender.com/v4/hash/{file_hash}"

        try:
            # Verificăm hash-ul în baza de date
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                scan_results = response.json().get("scan_results", {})
                detected = scan_results.get("total_detected_avs", 0)
                total = scan_results.get("total_avs", 0)
                data_id = response.json().get("data_id", "")
                analysis_url = f"https://www.metadefender.com/app/#!/file/{data_id}"
                analysis_url_good = f"https://metadefender.com/results/hash/{file_hash}"
                return (
                    f"Fișierul a fost găsit:\n"
                    f" - Detectat: {detected}/{total}\n"
                    f" - Detalii: {scan_results.get('scan_all_result_a', 'N/A')}\n"
                    f" - Link: {analysis_url_good}"
                )
            elif response.status_code == 404 and file_path:
                # Dacă hash-ul nu este găsit, încărcăm fișierul
                self.display_result("Hash-ul nu există. Încercare de încărcare a fișierului...")
                return self.upload_to_metadefender(file_path)
            else:
                return f"Eroare la interogarea MetaDefender. Status code: {response.status_code}\nRăspuns: {response.text}"
        except Exception as e:
            return f"Eroare la interogarea MetaDefender: {e}"




    def upload_to_virustotal(self, file_path):
        """
        Încarcă un fișier în VirusTotal pentru analiză și afișează Analysis ID și linkul.
        """
        api_key = config.get("settings", "virustotalapikey")
        headers = {"x-apikey": api_key}
        url = "https://www.virustotal.com/api/v3/files"

        try:
            with open(file_path, "rb") as file:
                files = {"file": (os.path.basename(file_path), file)}
                response = requests.post(url, headers=headers, files=files)

            if response.status_code == 200:
                analysis_id = response.json().get("data", {}).get("id", "")
                analysis_url = f"https://www.virustotal.com/gui/file-analysis/{analysis_id}"
                return f"Fișierul a fost încărcat cu succes.\nAnalysis ID: {analysis_id}\nLink: {analysis_url}"
            else:
                return f"Eroare la încărcarea fișierului. Status code: {response.status_code}\nRăspuns: {response.text}"
        except Exception as e:
            return f"Eroare la încărcarea fișierului: {e}"


    def upload_to_metadefender(self, file_path):
        """
        Încarcă un fișier în MetaDefender pentru analiză și returnează un hash pentru urmărire.
        """
        api_key = config.get("settings", "metadefenderapikey")
        headers = {"apikey": api_key}
        url = "https://api.metadefender.com/v4/file"

        try:
            with open(file_path, "rb") as file:
                response = requests.post(url, headers=headers, files={"file": file})

            if response.status_code == 200:
                file_hash = response.json().get("sha256", "")
                return f"Fișierul a fost încărcat cu succes. Puteți vizualiza analiza la:\nhttps://metadefender.com/results/hash/{file_hash}"
            else:
                return f"Eroare la încărcarea fișierului. Status code: {response.status_code}\nRăspuns: {response.text}"
        except Exception as e:
            return f"Eroare la încărcarea fișierului: {e}"




    

    def get_virustotal_analysis(self, analysis_id):
        """
        Obține rezultatele analizei unui fișier utilizând Analysis ID și afișează linkul.
        """
        api_key = config.get("settings", "virustotalapikey")
        if not api_key:
            return "Cheia API VirusTotal nu este configurată."

        headers = {"x-apikey": api_key}
        url = f"https://www.virustotal.com/api/v3/analyses/{analysis_id}"

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                stats = response.json().get("data", {}).get("attributes", {}).get("stats", {})
                analysis_url = f"https://www.virustotal.com/gui/file-analysis/{analysis_id}"
                return (
                    f"Analiză completă:\n"
                    f" - Malicious: {stats.get('malicious', 0)}\n"
                    f" - Suspicious: {stats.get('suspicious', 0)}\n"
                    f" - Link: {analysis_url}"
                )
            else:
                return f"Eroare la obținerea analizei VirusTotal. Status code: {response.status_code}\nRăspuns: {response.text}"
        except Exception as e:
            return f"Eroare la conectarea cu VirusTotal: {e}"


    def get_metadefender_analysis(self, file_hash):
        """
        Obține rezultatele analizei unui hash utilizând URL-ul specific MetaDefender.
        """
        api_key = config.get("settings", "metadefenderapikey")
        if not api_key:
            return "Cheia API MetaDefender nu este configurată."

        headers = {"apikey": api_key}
        url = f"https://metadefender.com/results/hash/{file_hash}"

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                # Răspunsul poate fi formatat diferit; ajustați dacă este necesar
                return f"Rezultate analiză:\n{response.json()}"
            else:
                return f"Eroare la obținerea analizei MetaDefender. Status code: {response.status_code}\nRăspuns: {response.text}"
        except Exception as e:
            return f"Eroare la conectarea cu MetaDefender: {e}"




    def create_widgets(self):
        # Secțiunea de fișier
        self.file_button = tk.Button(self.root, text="Selectează fișier", command=self.load_and_scan_file)
        self.file_button.pack(padx=10, pady=5)

        # Secțiunea de introducere hash
        self.hash_entry_label = tk.Label(self.root, text="Introdu un hash (SHA-256):")
        self.hash_entry_label.pack(pady=5)
        self.hash_entry = tk.Entry(self.root, width=64)
        self.hash_entry.pack(padx=10, pady=5)

        self.hash_check_button = tk.Button(self.root, text="Verifică hash-ul", command=self.check_input_hash)
        self.hash_check_button.pack(padx=10, pady=5)

        # Secțiunea de rezultate
        self.results_text = tk.Text(self.root, height=20, width=80)
        self.results_text.pack(padx=10, pady=10)
        self.results_text.config(state=tk.DISABLED)

    def check_input_hash(self):
        """
        Verifică hash-ul introdus de utilizator.
        """
        input_hash = self.hash_entry.get().strip()
        if not input_hash:
            self.display_result("Introduceți un hash valid.")
            return

        if len(input_hash) != 64 or not all(c in "0123456789abcdef" for c in input_hash.lower()):
            self.display_result("Hash-ul introdus nu este valid (trebuie să fie un SHA-256).")
            return

        self.display_result(f"Hash introdus: {input_hash}")
        self.check_hash(input_hash)

    def check_hash(self, file_hash):
        """
        Verifică hash-ul împotriva fișierelor locale și serviciilor API.
        """
        # Verificăm hash-ul în toate fișierele SHA256-Hashes_pack
        self.display_result("\nVerificare hash în fișierele hash:")
        hash_files = ["./hard_signatures/SHA256-Hashes_pack1.txt",
                    "./hard_signatures/SHA256-Hashes_pack2.txt",
                    "./hard_signatures/SHA256-Hashes_pack3.txt",
                    "./hard_signatures/MD5-Hashes.txt"]
        hash_found = False

        for hash_file in hash_files:
            self.display_result(f"Verificare în {os.path.basename(hash_file)}:")
            result = self.check_hash_in_file(file_hash, hash_file)
            self.display_result(result)
            if "a fost găsit" in result:
                hash_found = True

        if not hash_found:
            self.display_result("Hash-ul NU a fost găsit în niciun fișier de hash-uri.")

        # Verificări suplimentare (VirusTotal, MetaDefender)
        if self.use_virustotal_var.get():
            self.display_result("\nVerificare cu VirusTotal:")
            vt_result = self.check_virustotal(file_hash)
            self.display_result(vt_result)

        if self.use_metadefender_var.get():
            self.display_result("\nVerificare cu MetaDefender:")
            md_result = self.check_metadefender(file_hash)
            self.display_result(md_result)




# Inițiere aplicație
root = tk.Tk()
app = VirusScanApp(root)
root.mainloop()
